# Data Transformation Pipeline
This notebook demonstrates data cleaning, transformation, and preparation for analytics.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os
from datetime import datetime, timedelta
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

plt.style.use('default')
sns.set_palette('husl')

## 1. Load Raw Data
Load raw data from CSV files or database.

In [ ]:
def load_raw_data():
    """Load raw data from CSV files"""
    data_path = '../data/raw/'
    datasets = {}
    
    csv_files = ['customers.csv', 'transactions.csv', 'events.csv', 'products.csv']
    
    for file in csv_files:
        file_path = os.path.join(data_path, file)
        if os.path.exists(file_path):
            dataset_name = file.replace('.csv', '')
            datasets[dataset_name] = pd.read_csv(file_path)
            logger.info(f"Loaded {dataset_name}: {len(datasets[dataset_name])} records")
    
    return datasets

raw_data = load_raw_data()
print("Raw data loaded successfully!")
for name, df in raw_data.items():
    print(f"{name}: {df.shape}")

## 2. Data Cleaning
Handle missing values, duplicates, and data type conversions.

In [ ]:
def clean_customers_data(df):
    """Clean customer data"""
    df_clean = df.copy()
    
    # Convert date columns
    df_clean['registration_date'] = pd.to_datetime(df_clean['registration_date'])
    
    # Handle missing values
    df_clean['phone'].fillna('Unknown', inplace=True)
    
    # Standardize categorical data
    df_clean['state'] = df_clean['state'].str.upper()
    df_clean['segment'] = df_clean['segment'].str.title()
    
    # Create derived features
    df_clean['days_since_registration'] = (datetime.now() - df_clean['registration_date']).dt.days
    df_clean['age_group'] = pd.cut(df_clean['age'], bins=[0, 25, 35, 50, 65, 100], 
                                   labels=['18-25', '26-35', '36-50', '51-65', '65+'])
    
    return df_clean

def clean_transactions_data(df):
    """Clean transaction data"""
    df_clean = df.copy()
    
    # Convert date columns
    df_clean['transaction_date'] = pd.to_datetime(df_clean['transaction_date'])
    
    # Remove failed transactions for analysis
    df_clean = df_clean[df_clean['status'] != 'failed'].copy()
    
    # Create derived features
    df_clean['transaction_month'] = df_clean['transaction_date'].dt.month
    df_clean['transaction_day_of_week'] = df_clean['transaction_date'].dt.day_name()
    df_clean['transaction_hour'] = df_clean['transaction_date'].dt.hour
    
    # Categorize amount ranges
    df_clean['amount_category'] = pd.cut(df_clean['amount'], 
                                        bins=[0, 50, 200, 500, float('inf')],
                                        labels=['Small', 'Medium', 'Large', 'Very Large'])
    
    return df_clean

def clean_events_data(df):
    """Clean events data"""
    df_clean = df.copy()
    
    # Convert timestamp
    df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
    
    # Create derived features
    df_clean['event_date'] = df_clean['timestamp'].dt.date
    df_clean['event_hour'] = df_clean['timestamp'].dt.hour
    
    return df_clean

def clean_products_data(df):
    """Clean products data"""
    df_clean = df.copy()
    
    # Convert date columns
    df_clean['created_date'] = pd.to_datetime(df_clean['created_date'])
    
    # Calculate profit margin
    df_clean['profit_margin'] = (df_clean['price'] - df_clean['cost']) / df_clean['price'] * 100
    
    # Categorize stock levels
    df_clean['stock_level'] = pd.cut(df_clean['stock_quantity'],
                                    bins=[-1, 0, 50, 200, float('inf')],
                                    labels=['Out of Stock', 'Low', 'Medium', 'High'])
    
    return df_clean

# Clean all datasets
cleaned_data = {}
cleaned_data['customers'] = clean_customers_data(raw_data['customers'])
cleaned_data['transactions'] = clean_transactions_data(raw_data['transactions'])
cleaned_data['events'] = clean_events_data(raw_data['events'])
cleaned_data['products'] = clean_products_data(raw_data['products'])

print("Data cleaning completed!")
for name, df in cleaned_data.items():
    print(f"Cleaned {name}: {df.shape}")

## 3. Data Integration
Create integrated datasets by joining tables.

In [ ]:
def create_customer_transaction_summary():
    """Create customer transaction summary"""
    customers = cleaned_data['customers']
    transactions = cleaned_data['transactions']
    
    # Aggregate transaction data per customer
    txn_summary = transactions.groupby('customer_id').agg({
        'amount': ['count', 'sum', 'mean', 'std'],
        'transaction_date': ['min', 'max'],
        'currency': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'USD'
    }).round(2)
    
    # Flatten column names
    txn_summary.columns = ['_'.join(col).strip() for col in txn_summary.columns]
    txn_summary = txn_summary.rename(columns={
        'amount_count': 'total_transactions',
        'amount_sum': 'total_spent',
        'amount_mean': 'avg_transaction_amount',
        'amount_std': 'transaction_amount_std',
        'transaction_date_min': 'first_transaction',
        'transaction_date_max': 'last_transaction',
        'currency_<lambda>': 'primary_currency'
    })
    
    # Calculate days since last transaction
    txn_summary['days_since_last_transaction'] = (
        datetime.now() - pd.to_datetime(txn_summary['last_transaction'])
    ).dt.days
    
    # Join with customer data
    customer_summary = customers.merge(txn_summary, on='customer_id', how='left')
    
    # Fill NaN values for customers with no transactions
    txn_columns = ['total_transactions', 'total_spent', 'avg_transaction_amount']
    customer_summary[txn_columns] = customer_summary[txn_columns].fillna(0)
    
    return customer_summary

def create_daily_metrics():
    """Create daily business metrics"""
    transactions = cleaned_data['transactions']
    events = cleaned_data['events']
    
    # Daily transaction metrics
    daily_txn = transactions.groupby(transactions['transaction_date'].dt.date).agg({
        'amount': ['count', 'sum', 'mean'],
        'customer_id': 'nunique'
    }).round(2)
    
    daily_txn.columns = ['daily_transactions', 'daily_revenue', 'avg_transaction_value', 'unique_customers']
    daily_txn.reset_index(inplace=True)
    daily_txn.rename(columns={'transaction_date': 'date'}, inplace=True)
    
    # Daily event metrics
    daily_events = events.groupby(events['timestamp'].dt.date).agg({
        'event_id': 'count',
        'customer_id': 'nunique',
        'session_id': 'nunique'
    })
    
    daily_events.columns = ['daily_events', 'active_users', 'sessions']
    daily_events.reset_index(inplace=True)
    daily_events.rename(columns={'timestamp': 'date'}, inplace=True)
    
    # Merge metrics
    daily_metrics = daily_txn.merge(daily_events, on='date', how='outer').fillna(0)
    
    return daily_metrics

# Create integrated datasets
customer_summary = create_customer_transaction_summary()
daily_metrics = create_daily_metrics()

print(f"Customer summary created: {customer_summary.shape}")
print(f"Daily metrics created: {daily_metrics.shape}")

# Display sample data
print("\nCustomer Summary Sample:")
print(customer_summary[['customer_id', 'segment', 'total_transactions', 'total_spent', 'avg_transaction_amount']].head())

print("\nDaily Metrics Sample:")
print(daily_metrics.head())

## 4. Feature Engineering
Create advanced features for analytics and machine learning.

In [ ]:
def create_customer_segments():
    """Create RFM segmentation for customers"""
    transactions = cleaned_data['transactions']
    
    # Calculate RFM metrics
    current_date = transactions['transaction_date'].max()
    
    rfm = transactions.groupby('customer_id').agg({
        'transaction_date': lambda x: (current_date - x.max()).days,  # Recency
        'transaction_id': 'count',  # Frequency
        'amount': 'sum'  # Monetary
    })
    
    rfm.columns = ['recency', 'frequency', 'monetary']
    
    # Create quintiles for each metric
    rfm['r_score'] = pd.qcut(rfm['recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
    rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    rfm['m_score'] = pd.qcut(rfm['monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])
    
    # Combine scores
    rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)
    
    # Define segments based on RFM scores
    def segment_customers(row):
        if row['rfm_score'] in ['555', '554', '544', '545', '454', '455', '445']:
            return 'Champions'
        elif row['rfm_score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
            return 'Loyal Customers'
        elif row['rfm_score'] in ['512', '511', '422', '421', '412', '411', '311']:
            return 'Potential Loyalists'
        elif row['rfm_score'] in ['533', '532', '531', '523', '522', '521', '515', '514', '513', '425', '424', '413', '414', '415', '315', '314', '313']:
            return 'New Customers'
        elif row['rfm_score'] in ['155', '154', '144', '214', '215', '115', '114']:
            return 'At Risk'
        elif row['rfm_score'] in ['255', '254', '245', '244', '253', '252', '243', '242', '235', '234', '225', '224']:
            return 'Cannot Lose Them'
        else:
            return 'Others'
    
    rfm['segment'] = rfm.apply(segment_customers, axis=1)
    
    return rfm

def create_cohort_analysis():
    """Create cohort analysis data"""
    transactions = cleaned_data['transactions']
    
    # Get customer's first purchase date
    transactions['transaction_period'] = transactions['transaction_date'].dt.to_period('M')
    
    cohort_data = transactions.groupby('customer_id')['transaction_date'].min().reset_index()
    cohort_data.columns = ['customer_id', 'cohort_group']
    cohort_data['cohort_group'] = cohort_data['cohort_group'].dt.to_period('M')
    
    # Merge with transaction data
    df_cohort = transactions.merge(cohort_data, on='customer_id')
    df_cohort['period_number'] = (df_cohort['transaction_period'] - df_cohort['cohort_group']).apply(attrgetter('n'))
    
    # Create cohort table
    cohort_sizes = df_cohort.groupby('cohort_group')['customer_id'].nunique().reset_index()
    cohort_table = df_cohort.groupby(['cohort_group', 'period_number'])['customer_id'].nunique().reset_index()
    cohort_table = cohort_table.merge(cohort_sizes, on='cohort_group')
    cohort_table['retention_rate'] = cohort_table['customer_id_x'] / cohort_table['customer_id_y']
    
    return cohort_table.pivot(index='cohort_group', columns='period_number', values='retention_rate')

# Create advanced features
rfm_segments = create_customer_segments()

print("RFM Segmentation completed!")
print(rfm_segments['segment'].value_counts())
print("\nRFM Sample:")
print(rfm_segments.head())

## 5. Data Quality Validation
Validate transformed data quality.

In [ ]:
def validate_transformed_data():
    """Validate quality of transformed datasets"""
    validation_results = {}
    
    # Validate customer summary
    customer_issues = []
    if customer_summary['total_spent'].min() < 0:
        customer_issues.append("Negative spend amounts found")
    if customer_summary['total_transactions'].isnull().sum() > 0:
        customer_issues.append("NULL transaction counts found")
    
    validation_results['customer_summary'] = {
        'issues': customer_issues,
        'records': len(customer_summary),
        'status': 'PASS' if len(customer_issues) == 0 else 'FAIL'
    }
    
    # Validate daily metrics
    daily_issues = []
    if daily_metrics['daily_revenue'].min() < 0:
        daily_issues.append("Negative revenue found")
    if daily_metrics.isnull().sum().sum() > 0:
        daily_issues.append("NULL values found in metrics")
    
    validation_results['daily_metrics'] = {
        'issues': daily_issues,
        'records': len(daily_metrics),
        'status': 'PASS' if len(daily_issues) == 0 else 'FAIL'
    }
    
    # Validate RFM segments
    rfm_issues = []
    if rfm_segments.isnull().sum().sum() > 0:
        rfm_issues.append("NULL values in RFM scores")
    if len(rfm_segments['segment'].unique()) < 3:
        rfm_issues.append("Insufficient segment diversity")
    
    validation_results['rfm_segments'] = {
        'issues': rfm_issues,
        'records': len(rfm_segments),
        'status': 'PASS' if len(rfm_issues) == 0 else 'FAIL'
    }
    
    return validation_results

validation_results = validate_transformed_data()

print("Data Validation Results:")
for dataset, results in validation_results.items():
    print(f"\n{dataset.upper()}:")
    print(f"  Status: {results['status']}")
    print(f"  Records: {results['records']}")
    if results['issues']:
        print(f"  Issues: {', '.join(results['issues'])}")
    else:
        print("  Issues: None")

## 6. Save Processed Data
Save transformed data for analytics and modeling.

In [ ]:
def save_processed_data():
    """Save all processed datasets"""
    processed_path = '../data/processed/'
    
    # Ensure directory exists
    os.makedirs(processed_path, exist_ok=True)
    
    # Save all cleaned datasets
    for name, df in cleaned_data.items():
        file_path = os.path.join(processed_path, f'{name}_cleaned.csv')
        df.to_csv(file_path, index=False)
        logger.info(f"Saved {name}_cleaned.csv")
    
    # Save integrated datasets
    customer_summary.to_csv(os.path.join(processed_path, 'customer_summary.csv'), index=False)
    daily_metrics.to_csv(os.path.join(processed_path, 'daily_metrics.csv'), index=False)
    rfm_segments.to_csv(os.path.join(processed_path, 'rfm_segments.csv'))
    
    logger.info("All processed data saved successfully")
    
    # Create data catalog
    catalog = {
        'cleaned_datasets': {
            name: {'records': len(df), 'columns': len(df.columns)} 
            for name, df in cleaned_data.items()
        },
        'integrated_datasets': {
            'customer_summary': {'records': len(customer_summary), 'columns': len(customer_summary.columns)},
            'daily_metrics': {'records': len(daily_metrics), 'columns': len(daily_metrics.columns)},
            'rfm_segments': {'records': len(rfm_segments), 'columns': len(rfm_segments.columns)}
        },
        'transformation_date': datetime.now().isoformat()
    }
    
    import json
    with open(os.path.join(processed_path, 'data_catalog.json'), 'w') as f:
        json.dump(catalog, f, indent=2)
    
    return catalog

catalog = save_processed_data()
print("\nData Transformation Summary:")
print(f"Cleaned datasets: {len(catalog['cleaned_datasets'])}")
print(f"Integrated datasets: {len(catalog['integrated_datasets'])}")
print(f"Transformation completed: {catalog['transformation_date']}")